# COMS W1002 Computing in Context: Digital Humanities / Finance
## Schelling's  Model of Segregation
Adapted from Frank McCown and Stanford's Nifty Assignments repository! Many thanks to Frank and Nifty!

### Overview
Racial segregation has always been a pernicious social problem in the United States. Although much effort has been extended to desegregate our schools, churches, and neighborhoods, the [US continues to remain segregated](https://tcf.org/content/commentary/racial-segregation-is-still-a-problem/) by race and economic lines. Why is segregation such a difficult problem to eradicate?

In 1971, the American economist [Thomas Schelling](https://en.wikipedia.org/wiki/Thomas_Schelling) created an agent-based model that might help explain why segregation is so difficult to combat. His model of segregation showed that even when individuals (or "agents") didn't mind being surrounded or living by agents of a different race, they would still choose to segregate themselves from other agents over time! Although the model is quite simple, it gives a fascinating look at how individuals might self-segregate, even when they have no explicit desire to do so.

In this assignment, you will create a simulation of Schelling's model. The user should be able to set a number of parameters of the model and watch it go.

### How the Model Works

Schelling's model will now be explained with some minor changes. Suppose there are two types of agents: X and O. The two types of agents might represent different races, ethnicity, economic status, etc. Two populations of the two agent types are initially placed into random locations of a neighborhood represented by a grid. After placing all the agents in the grid, each cell is either occupied by an agent or is empty as shown below. 

![alt text](http://www.cs.columbia.edu/~cannon/images/grid1.png "Agents placed randomly")

Now we must determine if each agent is satisfied with its current location. A satisfied agent is one that is surrounded by at least a fractoin *t* of agents that are like itself. This threshold *t* is one that will apply to all agents in the model, even though in reality everyone might have a different threshold they are satisfied with. Note that the higher the threshold, the higher the likelihood the agents will not be satisfied with their current location.

For example, if *t = .3*, agent X is satisfied if at least 30% of its neighbors are also X. If fewer than 30% are X, then the agent is not satisfied, and it will want to change its location in the grid. For the remainder of this explanation, let's assume a threshold *t* of .3. This means every agent is fine with being in the minority as long as there are at least 30% of similar agents in adjacent cells.

The picture below shows a satisfied agent because 50% of X's neighbors are also X (.5 > t). 

![alt text](http://www.cs.columbia.edu/~cannon/images/grid2.png "Satisfied agent") 

In the pciture below, agent X is not satisfied because only 25% of its neighbors are X (.25 < t). Notice that in this example empty cells are not counted when calculating similarity.

![alt text](http://www.cs.columbia.edu/~cannon/images/grid3.png "Dissatisfied agent")

When an agent is not satisfied, it can be moved to any vacant location in the grid. Any algorithm can be used to choose this new location. For example, a randomly selected cell may be chosen, or the agent could move to the nearest available location.

In the image below, all dissatisfied agents have an asterisk next to them. 


![alt text](http://www.cs.columbia.edu/~cannon/images/grid4.png "Dissatisfied agents with asterisk") 

The next image shows the new configuration after all the dissatisfied agents have been moved to unoccupied cells at random. Note that the new configuration may cause some agents which were previously satisfied to become dissatisfied! 

![alt text](http://www.cs.columbia.edu/~cannon/images/grid5.png "New configuration") 

All dissatisfied agents must be moved in the same round. After the round is complete, a new round begins, and dissatisfied agents are once again moved to new locations in the grid. These rounds continue until all agents in the neighborhood are satisfied with their location.

### Okay, so what do I have to do?

You will create code that simulates the Schelling model for two distinct populations.
Consider the following parameters for the Schelling model:
* grid size: tuple of integers (n,m)
* fraction of population 1: float between 0 and 1.
* fraction of vacant sites in the grid: float betweein 0 and 1.
* tolerance: float between 0 and 1

Write a function `start(grid_size,pop,vacancy)` that creates and returns a 2-D numpy array of the appropriate size consisting of integers where each element of the array is assigned as follows:
* part of first demographic:  1
* part of second demographic: -1
* vacant: 0

The array your function returns should have the appropriate characteristics as specified by the input parameters provided.


In [98]:
# write your function here:
def start(grid_size,pop,vacancy):
    '''This function makes a population grid'''
    import numpy as np
    import random
    
    # get the size of the grid
    size = grid_size[0]*grid_size[1]
    
    # number of vacant spaces
    num_vac = int(vacancy*size)
    
    # number of population 2
    num_pop2 = int((1-pop)*(size-num_vac))
    
    # create the grid
    grid = np.ones(size)
    
    # check for valid user entry
    if ((pop*(1-vacancy))+vacancy) > 1:
        return ("Please make sure that you enter valid percentages.")
    else:
        # put the correct number of vacancies and members of each pop
        grid[0:num_vac] = 0
        grid[num_vac:num_vac+num_pop2] = -1
        # randomize grid
        grid = np.random.permutation(grid)
        # reshape
        grid.shape = grid_size
        
    return grid

### But wait, there's more!

Write a second function `next_round(arr,tolerance)` that takes as input an *n by m* array like the one you just created and returns a new 2-D array one step in the future where all dissatisfied agents have been moved. Remember, since there may be more dissatisfied agents than vacancies in the grid, you need to move them one at a time, this way each time an agent is randomly assigned to a vacant site, a vacancy at a new location is created.

In [95]:
def is_satisfied(arr, i, j, tolerance):
    '''returns a boolean which tells us if the individual at i,j is satisfied'''
    
    # keep track of all the neighbors(denom) and the neighbors of the same pop(num)
    num = -1
    denom = -1
    
    #case1 - first row
    if i == 0:
        # first column
        if j == 0:
            for x in range(i, i+2):
                for y in range(j, j+2):
                    # don't count vacancies in the total neighbors
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
            
        elif j == arr.shape[1]-1:
            # last column
            for x in range(i, i+2):
                for y in range(j-1, j+1):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num +=1
        else:
            # middle
            for x in range(i, i+2):
                for y in range(j-1, j+2):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num +=1
                            
    # last row                    
    elif i == arr.shape[0] - 1:
        if j == 0:
            # first column
            for x in range(i-1, i+1):
                for y in range(j, j+2):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
            
        elif j == arr.shape[1]-1:
            # last column
            for x in range(i-1, i+1):
                for y in range(j-1, j+1):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
        else:
            # middle
             for x in range(i-1, i+1):
                for y in range(j-1, j+2):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
    # middle row                       
    else:
        if j == 0:
            # first column
            for x in range(i-1, i+2):
                for y in range(j, j+2):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
            
        elif j == arr.shape[1]-1:
            # last column
             for x in range(i-1, i+2):
                for y in range(j-1, j+1):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
        else:
            # middle
             for x in range(i-1, i+2):
                for y in range(j-1, j+2):
                    if arr[x][y] != 0:
                        denom += 1
                        if arr[x][y] == arr[i][j]:
                            num+=1
                            
    # find the fraction, determine if it's over the tolerance 
    if (denom == 0) or (num / denom >= tolerance):
        return True
    else:
        return False
    
def next_round(arr,tolerance):
    import numpy as np
    import random
    
    # finds empty spaces
    where_z = np.argwhere(arr == 0)
    
    # copy the array so we don't change the old one
    new_arr = np.copy(arr)
    
    # traverse the array and see whether each person is satisfied
    for i in range(0, arr.shape[0]):
        for j in range(0, arr.shape[1]):
            move = False
            if arr[i][j] != 0:
                
                # move if the individual is not satisfied
                move = not(is_satisfied(arr,i,j,tolerance))
            if move:
                
                # look for empty spaces and choose one randomly 
                location = random.randint(0, len(where_z)-1)
                
                # change the zero to the value of your person, change your person to zero
                new_arr[where_z[location][0], where_z[location][1]] = arr[i][j]
                new_arr[i][j] = 0
                
                # indicate that the vacant spot is now the spot of the person who just moved
                where_z[location] = np.array([i,j])
    
    return new_arr



### Last bit! (almost)

Finally write a function `make_list(n)` that returns a list of *n* numpy arrays where the first array is created using `start` and the second using `next_round` on the first array and the third using `next_round` on the second, and so on. 

In [96]:
def make_list(n,grid_size,pop,vacancy,tolerance):
    '''make a list of n numpy arrays using start and next_round'''
    import numpy as np
    
    # make the grid and the list
    grid = start(grid_size, pop, vacancy)
    grid_list = [grid]
    
    # add every next_round array to the list
    for t in range(0,n):
        grid_list.append(next_round(grid, tolerance))
    return(grid_list)

def make_list2(grid_size, pop, vacancy, tolerance):
    '''make a list of numpy arrays until all people are satisfied'''
    import numpy as np
    
    g1 = start(grid_size, pop, vacancy)
    g2 = next_round(g1, tolerance)
    grid_list = [g1]
    
    # while not satisfied, keep adding arrays to the list
    while (not(np.array_equal(g1, g2))):
        grid_list.append(g2)
        g1 = g2
        g2 = next_round(g2, tolerance)
    return(grid_list)
    

### Really last bit!

Try out your code. Create a list of length 100 using the parameter values:
* size=(30,30)
* pop=.5
* vacant = .1
* tolerance = .3

In class we will see how to visualize this evolution of the grid. Try it out! 

And finally, are there still dissatisfied agents in the grid? How could you write an alternative function to `make_list` that makes a list exactly big enough so that in the final grid there are no more dissatisfied agents. You don't have to do this, but it wouldn't be too hard.



In [99]:
t = make_list(100, (30,30), 0.5, 0.10, 0.30)
len(make_list2((30,30), 0.5, 0.10, 0.30))

16